<a href="https://colab.research.google.com/github/narbouk-afk/Predict_student_performance/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### links : 

https://github.com/narbouk-afk/Predict_student_performance

https://www.kaggle.com/competitions/predict-student-performance-from-game-play

# Import

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data import

## Complete Dataframe

In [4]:
path = '/content/drive/My Drive/Dev/KaggleFDA/'
train = pd.read_csv(path +'train.csv')
features = [c for c in train.columns ]
train['split'] = 'train'
test = pd.read_csv(path +'test.csv')
test['split'] = 'test'
# data = pd.concat([train,test]).reset_index(drop=True)

In [5]:
train_labels =  pd.read_csv(path +'train_labels.csv')

In [6]:
# data.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13174211 entries, 0 to 13174210
Data columns (total 21 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      int64  
 1   index           int64  
 2   elapsed_time    int64  
 3   event_name      object 
 4   name            object 
 5   level           int64  
 6   page            float64
 7   room_coor_x     float64
 8   room_coor_y     float64
 9   screen_coor_x   float64
 10  screen_coor_y   float64
 11  hover_duration  float64
 12  text            object 
 13  fqid            object 
 14  room_fqid       object 
 15  text_fqid       object 
 16  fullscreen      float64
 17  hq              float64
 18  music           float64
 19  level_group     object 
 20  split           object 
dtypes: float64(9), int64(4), object(8)
memory usage: 2.1+ GB


In [7]:
# data.head()
train.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,split
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,train
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,train
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,train
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,train
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,train


In [8]:
# data.shape
train.shape

(13174211, 21)

# Preprocesing

## Modifications on the data, train test

Modification to make it easier to use the data set

## Tran labels + question column

This bloc is to access the labels with the results more easily

In [9]:
train_labels.shape

(212022, 2)

In [10]:
train_labels_question = train_labels.copy()
train_labels_question[['session_id','question']] = train_labels_question['session_id'].str.split('_', expand=True)
train_labels_question.head()

,session_id,correct,question
0,20090312431273200,1,q1
1,20090312433251036,0,q1
2,20090314121766812,1,q1
3,20090314363702160,1,q1
4,20090314441803444,1,q1


## Feature engineering (if no DL)

## Missing values 

We will have to handle missing values differently according to the type.

In [11]:
print(train.isna().sum())

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page              12889465
room_coor_x        1036240
room_coor_y        1036240
screen_coor_x      1036240
screen_coor_y      1036240
hover_duration    12173474
text               8367320
fqid               4159648
room_fqid                0
text_fqid          8367266
fullscreen        13174211
hq                13174211
music             13174211
level_group              0
split                    0
dtype: int64


The missing values in the text and text_fqid variables come from the fact that a text doesn't always appear at each

The missing values in the fqid variables come from the fact that a descrition isn't mandatory for every event


For the room_coor x, y, screen_coor x and y there are exactly the same number of missing values which is normal because they only appear in click event

In [12]:
# count_na_coord = train['event_name'].str.contains('click').sum()
# print(len(train)- count_na_coord)
# = 1036240

Similarally, the page and hoover duration have missing values because they depend on a sppesific type of event (r. notebook related events and hover events)

Fullscreen, hq and usic have no value, we can drop them

In [13]:
train.drop(['hq','music','fullscreen'],axis=1)
test.drop(['hq','music','fullscreen'],axis=1)
train.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,level_group,session_level,split
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,259.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4,20090109393214576_0-4,test
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,398.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,20090109393214576_0-4,test
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,287.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,20090109393214576_0-4,test
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,392.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,20090109393214576_0-4,test
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,392.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,20090109393214576_0-4,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,484.0,NaN,tunic.wildlife,tunic.historicalsociety.entry,NaN,13-22,20090312331414616_13-22,test
3724,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,783.0,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,13-22,20090312331414616_13-22,test
3725,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726363,-3.880047,456.0,332.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,13-22,20090312331414616_13-22,test
3726,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372139,38.216178,383.0,272.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22,20090312331414616_13-22,test


## Outliers

# Visualization 

# Dimension reduction